<a href="https://colab.research.google.com/github/petermr/dictionary/blob/main/interactive_map/geotext_geopy_map_oil186.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# install all the required packages
!pip install geopy
!pip install folium
!pip install geotext

In [51]:
# geotext gives the city names from parsed text
import pandas as pd
from geotext import GeoText
import re 
cities = []
df = pd.read_csv("oil186_20210712.csv", usecols = ["sentences"])
for index, row in df.iterrows():
    row["sentences"]=re.sub('\[', '', row["sentences"]) 
    row["sentences"]=re.sub('\]', '', row["sentences"])
    places = GeoText(str(row["sentences"]))
    cities.append(places.cities)
    df["cities"] = pd.Series(cities)


In [52]:
# geopy gives the coordinates of the city names using Nominatim (which is free and doesn't require API key)
from geopy.geocoders import Nominatim
import folium
geolocator = Nominatim(user_agent="snh")
lat = []
long = []
for index, row in df.iterrows():
    ct = geolocator.geocode(row["cities"], timeout=10)
    if ct is None:
        lat.append(None)
        long.append(None)
    else:
        latitude = (ct.latitude)
        longitude =(ct.longitude)
        lat.append(latitude)
        long.append(longitude)

df["lat"] = lat
df["long"]= long



In [53]:
#removes rows with empty list elements from the dataframe
df = df[df['cities'].astype(bool)]
df.head()

,sentences,cities,lat,long
3,'The species were collected in the city of Mac...,[Macapá],0.040153,-51.056959
6,"'Whole plants were collected from Keranigonj, ...",[Dhaka],23.810651,90.412647
7,"'The plant was collected from herbal farm, Mys...",[Mysore],12.305183,76.655361
12,'officinalis L. were collected in Chott Marie...,[March],48.057857,7.779004
14,'Mez were collected in Caxiuanã National Fores...,[Belém],-1.450560,-48.468245


In [58]:
# plotting on a world map
import folium
from folium.plugins import MarkerCluster
#empty map
locations = df[['lat', 'long']]
# droping NaNs
locations=locations.dropna()

locationlist = locations.values.tolist()
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)

df2=df.drop(['lat','long'], axis=1)

#for each coordinate, create circlemarker 
for i in range(0, len(locationlist)):
        #radius=5
        df1=df2.iloc[i].to_frame()
        folium.Marker(locationlist[i],popup= df1.to_html(),icon=folium.Icon(color='darkblue',icon= 'default',prefix='fa')).add_to(world_map)
        #folium.CircleMarker(locationlist[i], radius=radius, popup= df1.to_html(), fill =True, ).add_to(marker_cluster)

world_map.save('worldmap_2.html')


In [64]:
df

,sentences,cities,lat,long
3,'The species were collected in the city of Mac...,[Macapá],0.040153,-51.056959
6,"'Whole plants were collected from Keranigonj, ...",[Dhaka],23.810651,90.412647
7,"'The plant was collected from herbal farm, Mys...",[Mysore],12.305183,76.655361
12,'officinalis L. were collected in Chott Marie...,[March],48.057857,7.779004
14,'Mez were collected in Caxiuanã National Fores...,[Belém],-1.450560,-48.468245
19,'Fruits (pods) of A. melegueta were collect...,[Bafoussam],5.475808,10.421556
20,'Leaves of M. urundeuva were collected from...,"[University, Fortaleza]",-3.748131,-38.550820
22,'Four different samples of Allium vineale w...,[Huntsville],34.729847,-86.585901
26,'Leaves of V. guianensis were collected in ...,"[Camaçari, March]",NaN,NaN
27,"'The leaf, stem, seed, rhizome, and pod of A...",[March],48.057857,7.779004
